### Creating multi-agent collaboration: Hedge Fund assistant
---

In this folder we will create the supervisor agent for our `Hedge Fund assistant` and attach the `fundamental analyst assistant`, `technical analyst assistant` and `marketing analyst assistant` as sub-agents to it.

**IMPORTANT: This notebook uses the new `Amazon Nova Micro` model as the FM that powers the supervisor agent to route requests efficiently between associated sub agents.**

Each of the sub agents have access to three tools each that can access `APIs` to fetch financial data on ticker symbols and search the web. Based on the user question, the supervisor agent hedge assistant will be able to call the appropriate sub agent. The sub agent will then identify the question and call one of the functions that it has access to, and return the answer back to the user. View the diagram of what this looks like below:

![multi-agent-diagram](multi-agent-diagram.png)

This agent will make usage of the `SUPERVISOR_ROUTER` collaboration mode in order to get the user's intent classification routed to the correct sub-agent

### Agent invocation tracking
---

As a part of the `invoke_agent_helper` function, we have integrated tracking on `weave`. For this, enter your `WEAVE_API_KEY` in the `.env` file. View an example below:

![weave-tracking](weave-tracking.png)

### Prerequisites
---

You are going to install boto3 dependencies from pip. Make sure you have the latest version of it for full capabilities

Before running this notebook, make sure to populate the `.env` file with your `FINANCIAL_DATASETS_API_KEY` and `TAVILY_API_KEY`. These two `API` keys will be used by the agent to access information on user related questions. You can create an `API` here: https://www.financialdatasets.ai/

In [1]:
!pip uninstall boto3 botocore awscli --yes

Found existing installation: boto3 1.35.83
Uninstalling boto3-1.35.83:
  Successfully uninstalled boto3-1.35.83
Found existing installation: botocore 1.35.83
Uninstalling botocore-1.35.83:
  Successfully uninstalled botocore-1.35.83
Found existing installation: awscli 1.36.24
Uninstalling awscli-1.36.24:
  Successfully uninstalled awscli-1.36.24


In [2]:
# Install latest boto3
!python3 -m pip install --force-reinstall --no-cache -q --no-dependencies -r ../requirements.txt

### Import required libraries
---

Next we will import the required libraries. We will also import some support functions available in the parent directory. Those functions are:

1. `create_agent`: helps you to create the necessary IAM permissions and Bedrock agents based on the agent's name, instructions, foundation models, descriptions and other properties.

1. `associate_sub_agents`: associate existing agents as sub-agents to the created supervisor agent

1. `list_agent_collaborators`: list the available agents collaborations

1. `invoke_agent_helper`: helps you to invoke your agent using invoke_agent
You can see the implementation of both functions in the parent directory

In [3]:
import os
import sys
import json
import time
import logging

# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from globals import *
from agents import create_agent, associate_sub_agents, list_agent_collaborators, invoke_agent_helper

/Users/madhurpt/Desktop/hedge-fund-analyst-multi-agent-collaboration-2
Boto3 version: 1.35.83


[2024-12-17 19:49:07,972] p90570 {requests.py:227} INFO - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
[2024-12-17 19:49:08,065] p90570 {requests.py:263} INFO - <<< {"data":{"viewer":{"username":"madhur-prash","defaultEntity":{"name":"madhur-prash-none"}}}}
[2024-12-17 19:49:09,636] p90570 {utils.py:160} INFO - NumExpr defaulting to 8 threads.
[2024-12-17 19:49:10,528] p90570 {_client.py:1026} INFO - HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"
[2024-12-17 19:49:11,085] p90570 {config.py:59} INFO - PyTorch version 2.3.0 available.
[2024-12-17 19:49:11,320] p90570 {requests.py:227} INFO - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
[2024-12-17 19:49:11,410] p90570 {requests.py:263} INFO - <<< {"data":{"viewer":{"username":"madhur-prash","defaultEntity":{"name":"madhur-pr

Logged in as Weights & Biases user: madhur-prash.
View Weave data at https://wandb.ai/madhur-prash-none/multi-agent-collaboration/weave


In [4]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

#### Defining supervisor agent configuration

Let's now define the configuration for our hedge fund assistant supervisor agent. Let's use the follow instructions to create our agent:

You are a hedge fund assistant who fetches information on fundamental, technical and marketing analysis on financial data. You can fetch this different kind of financial data based on user questions.

In [5]:
%store -r
fundamental_analyst_agent_alias_arn, marketing_agent_alias_arn, technical_analyst_agent_alias_arn

('arn:aws:bedrock:us-east-1:218208277580:agent-alias/WH3Y06UPJI/4JZJVMDGU6',
 'arn:aws:bedrock:us-east-1:218208277580:agent-alias/WBUINUADYX/65H2UJP7TF',
 'arn:aws:bedrock:us-east-1:218208277580:agent-alias/BRU2RYCZR1/X0OOOOEXXL')

In [6]:
agent_instruction: str = """You are a sophisticated financial analysis system with access to three specialized analysts: a Fundamental Analyst who analyzes financial statements, balance sheets, and cash flows; 
a Technical Analyst who examines price patterns, technical indicators, and market trends; and a Sentiment Analyst who monitors market sentiment through options data, insider trading, and news coverage. When responding to user queries, 
coordinate with these analysts to provide comprehensive investment insights. The Fundamental Analyst will focus on company financials and valuation metrics, the Technical Analyst will analyze price movements and technical indicators, 
and the Sentiment Analyst will gauge market psychology and news sentiment. Work together to synthesize their individual analyses into a cohesive, well-rounded response that considers multiple perspectives on the investment opportunity. 
Each analysis should be clearly labeled and contribute to an overall recommendation or insight."""

In [7]:
agent_name = 'multi-agent-hedge-fund'
agent_description = "Multi-agent collaboration for hedge fund assistant to get fundamental, technical and marketing analysis"

sub_agents_list = [
    {
        'sub_agent_alias_arn': fundamental_analyst_agent_alias_arn,
        'sub_agent_instruction': """Use this agent when the user asks to analyze company financial statements across three key areas: income statements, balance sheets, and cash flow statements. You require the user to provide a stock ticker symbol to analyze the company's financials. If a user does not provide a ticker symbol, please ask for it as a follow-up question.""",
        'sub_agent_association_name': 'fundamental-analyst-agent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': marketing_agent_alias_arn,
        'sub_agent_instruction': """Use this agent when a user asks to analyze options chains, insider trading data, and relevant market news. You require the user to provide a stock ticker symbol for analysis. If a user does not provide a ticker symbol, mention in the answer that they need to provide a ticker symbol.""",
        'sub_agent_association_name': 'market-analysis-agent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': technical_analyst_agent_alias_arn,
        'sub_agent_instruction': """Use this agent if a users asks to analyze stock price movements and technical indicators. You can calculate and interpret various technical indicators using historical price data.""",
        'sub_agent_association_name': 'technical-analyst-agent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

### Create the hedge fund agent
---

Next we will create the agent with the provided information

In [10]:
supervisor_agent_id, supervisor_agent_alias_id, supervisor_agent_alias_arn = create_agent(
    agent_name,
    agent_instruction,
    agent_foundation_model=BEDROCK_NOVA_MICRO,
    agent_description=agent_description,
    agent_collaboration='SUPERVISOR',
    create_alias=False
)
supervisor_agent_id, supervisor_agent_alias_id, supervisor_agent_alias_arn

creating agent


('HM2GAT6PKT', 'TSTALIASID', None)

### Create agent associations
---

Next we will associate the required sub-agents to our supervisor agent

In [12]:
supervisor_agent_alias_id, supervisor_agent_alias_arn = associate_sub_agents(
    supervisor_agent_id, sub_agents_list
)
supervisor_agent_alias_id, supervisor_agent_alias_arn

Agent id HM2GAT6PKT current status: NOT_PREPARED
Agent id HM2GAT6PKT current status: NOT_PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id HM2GAT6PKT current status: PREPARED
Agent id HM2GAT6PKT current status: PREPARED
Agent id HM2GAT6PKT current status: PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id HM2GAT6PKT current status: PREPARED
Agent id HM2GAT6PKT current status: PREPARED
Agent id HM2GAT6PKT current status: PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id HM2GAT6PKT current status: PREPARED


('K3WDMSOCAT',
 'arn:aws:bedrock:us-east-1:218208277580:agent-alias/HM2GAT6PKT/K3WDMSOCAT')

Now you would have the four agents created and prepared on the console - this includes the `SUPERVISOR HEDGE FUND AGENT` and 3 `SUB AGENTS`

![agents](agents-created.png)

### List agent associations
---

After that we can list the associations of our agent

In [13]:
logger.info(f"The collaborators of the multi hedge fund agent are: {list_agent_collaborators(supervisor_agent_id)}")
logger.info(f"Number of agent collaborators: {len(list_agent_collaborators(supervisor_agent_id))}")

[2024-12-17 19:50:51,787] p90570 {1706908712.py:1} INFO - The collaborators of the multi hedge fund agent are: {'ResponseMetadata': {'RequestId': '63335227-c6dd-423d-8daf-afe32bdb0697', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 18 Dec 2024 00:50:51 GMT', 'content-type': 'application/json', 'content-length': '2024', 'connection': 'keep-alive', 'x-amzn-requestid': '63335227-c6dd-423d-8daf-afe32bdb0697', 'x-amz-apigw-id': 'C9lg5GtioAMECJA=', 'x-amzn-trace-id': 'Root=1-67621c6b-099054a63dee4b1f574bcb67'}, 'RetryAttempts': 0}, 'agentCollaboratorSummaries': [{'agentDescriptor': {'aliasArn': 'arn:aws:bedrock:us-east-1:218208277580:agent-alias/WH3Y06UPJI/4JZJVMDGU6'}, 'agentId': 'HM2GAT6PKT', 'agentVersion': 'DRAFT', 'collaborationInstruction': "Use this agent when the user asks to analyze company financial statements across three key areas: income statements, balance sheets, and cash flow statements. You require the user to provide a stock ticker symbol to analyze the company's fin

### Testing Agent
---

Now that we've created the agent, let's test it by using our invoke_agent_helper function

In [14]:
time.sleep(10)

#### Route to the Marketing analyst sub agent
---

In this portion, we will invoke the supervisor agent to look for General Council's stocks in `AAPL`. For this, the `marketing analyst agent` should be invoked.

The `get_news()` function is used by the `sub-agent` to get the latest information on the question below.

In [16]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """How many shares of AAPL has General Counsel sold over the past year?"""
response = invoke_agent_helper(query, session_id, supervisor_agent_id, supervisor_agent_alias_id, enable_trace=True)
print(response)

[2024-12-17 19:51:25,123] p90570 {agents.py:760} INFO - invoke_agent response metadata:
[2024-12-17 19:51:25,124] p90570 {agents.py:761} INFO - {
  "RequestId": "f2d5afe1-326e-451f-8e3f-d222e2e4452f",
  "HTTPStatusCode": 200,
  "HTTPHeaders": {
    "date": "Wed, 18 Dec 2024 00:51:25 GMT",
    "content-type": "application/vnd.amazon.eventstream",
    "transfer-encoding": "chunked",
    "connection": "keep-alive",
    "x-amzn-requestid": "f2d5afe1-326e-451f-8e3f-d222e2e4452f",
    "x-amz-bedrock-agent-session-id": "34ffdaa4-bcda-11ef-9404-168aeb12c117",
    "x-amzn-bedrock-agent-content-type": "application/json"
  },
  "RetryAttempts": 0
}
[2024-12-17 19:51:25,175] p90570 {agents.py:787} INFO - ==== Trace Event Received ====
[2024-12-17 19:51:25,176] p90570 {agents.py:790} INFO - callerChain: [
  {
    "agentAliasArn": "arn:aws:bedrock:us-east-1:218208277580:agent-alias/HM2GAT6PKT/K3WDMSOCAT"
  }
]
[2024-12-17 19:51:25,176] p90570 {agents.py:803} INFO - Orchestration Trace Detected
[2024

🍩 https://wandb.ai/madhur-prash-none/multi-agent-collaboration/r/call/0193d73f-86ba-7e02-b7de-23fd8b075fe6
To provide the information you're looking for, I need the stock ticker symbol of the company in question. Could you please provide the ticker symbol for the company?
CPU times: user 41.7 ms, sys: 8.78 ms, total: 50.5 ms
Wall time: 1.49 s


#### Route to the Fundamental analyst agent
---

In this portion, we will invoke the supervisor agent to look for AAPL's balance sheet information for the first quarter of May, 2021. For this, the supervisor agent should route the request to the `fundamental analyst sub agent` and get the information.

The `get_balance_sheets()` function is used by the sub agent to get the response below

In [17]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """What is AAPL's balance sheet information for 2023?"""
response = invoke_agent_helper(query, session_id, supervisor_agent_id, supervisor_agent_alias_id, enable_trace=True)
print(response)

[2024-12-17 19:51:30,239] p90570 {agents.py:760} INFO - invoke_agent response metadata:
[2024-12-17 19:51:30,239] p90570 {agents.py:761} INFO - {
  "RequestId": "769fec5f-84d4-464c-bf1e-7f7579610c64",
  "HTTPStatusCode": 200,
  "HTTPHeaders": {
    "date": "Wed, 18 Dec 2024 00:51:30 GMT",
    "content-type": "application/vnd.amazon.eventstream",
    "transfer-encoding": "chunked",
    "connection": "keep-alive",
    "x-amzn-requestid": "769fec5f-84d4-464c-bf1e-7f7579610c64",
    "x-amz-bedrock-agent-session-id": "380f2858-bcda-11ef-9404-168aeb12c117",
    "x-amzn-bedrock-agent-content-type": "application/json"
  },
  "RetryAttempts": 0
}
[2024-12-17 19:51:30,391] p90570 {agents.py:787} INFO - ==== Trace Event Received ====
[2024-12-17 19:51:30,392] p90570 {agents.py:790} INFO - callerChain: [
  {
    "agentAliasArn": "arn:aws:bedrock:us-east-1:218208277580:agent-alias/HM2GAT6PKT/K3WDMSOCAT"
  }
]
[2024-12-17 19:51:30,392] p90570 {agents.py:803} INFO - Orchestration Trace Detected
[2024

🍩 https://wandb.ai/madhur-prash-none/multi-agent-collaboration/r/call/0193d73f-9ac7-7a51-b423-00569459e371
Here is the balance sheet information for Apple Inc. (AAPL) for the 2023 fiscal year:

**Total Assets:** $364,980,000,000
- **Current Assets:** $152,987,000,000
  - Cash and Equivalents: $29,943,000,000
  - Inventory: $7,286,000,000
  - Current Investments: $35,228,000,000
  - Receivables: $66,243,000,000
- **Non-Current Assets:** $211,993,000,000
  - Property, Plant & Equipment: $45,680,000,000
  - Investments: $126,707,000,000

**Total Liabilities:** $308,030,000,000
- **Current Liabilities:** $176,392,000,000
  - Current Debt: $20,879,000,000
  - Payables: $68,960,000,000
  - Deferred Revenue: $8,249,000,000
- **Non-Current Liabilities:** $131,638,000,000
  - Non-Current Debt: $85,750,000,000

**Shareholders' Equity:** $56,950,000,000
- Retained Earnings: -$19,154,000,000
- Accumulated Other Comprehensive Income: -$7,172,000,000
- Outstanding Shares: 15,204,137,000

**Total Deb

### Route to the `Marketing analyst agent` to get news by searching on web
---

In [19]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """Can you give any news on 'AAPL' and what is its stock like? Search for 'Apple stock data'. Look for statistical data to provide. Be concise and only search once."""
response = invoke_agent_helper(query, session_id, supervisor_agent_id, supervisor_agent_alias_id, enable_trace=True)
print(response)

[2024-12-17 19:52:04,920] p90570 {agents.py:760} INFO - invoke_agent response metadata:
[2024-12-17 19:52:04,921] p90570 {agents.py:761} INFO - {
  "RequestId": "0f798b62-4d89-4461-b9d6-bf6868c92628",
  "HTTPStatusCode": 200,
  "HTTPHeaders": {
    "date": "Wed, 18 Dec 2024 00:52:04 GMT",
    "content-type": "application/vnd.amazon.eventstream",
    "transfer-encoding": "chunked",
    "connection": "keep-alive",
    "x-amzn-requestid": "0f798b62-4d89-4461-b9d6-bf6868c92628",
    "x-amz-bedrock-agent-session-id": "4cb53d9c-bcda-11ef-9404-168aeb12c117",
    "x-amzn-bedrock-agent-content-type": "application/json"
  },
  "RetryAttempts": 0
}
[2024-12-17 19:52:05,093] p90570 {agents.py:787} INFO - ==== Trace Event Received ====
[2024-12-17 19:52:05,093] p90570 {agents.py:790} INFO - callerChain: [
  {
    "agentAliasArn": "arn:aws:bedrock:us-east-1:218208277580:agent-alias/HM2GAT6PKT/K3WDMSOCAT"
  }
]
[2024-12-17 19:52:05,094] p90570 {agents.py:803} INFO - Orchestration Trace Detected
[2024

🍩 https://wandb.ai/madhur-prash-none/multi-agent-collaboration/r/call/0193d740-2219-7b02-8f64-602e9de204c5


Exception: ('unexpected event.', EventStreamError('An error occurred (dependencyFailedException) when calling the InvokeAgent operation: Agent technical-analyst-agent failed to process the request with error Dependency resource: received model timeout/error exception from Bedrock. Try the request again. (Service: bedrock, Status Code: 424, Request ID: b24805aa-264e-4fb3-b48d-fc8083fc8297)'))

### Invoking collaboration between two sub agents via one user query
---

In this example, we will ask for some news as well as balance sheet information which will require the supervisor agent to collaborate with both, the marketing analyst agent and the fundamental analyst agent.

In [20]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """Search for 'Apple stock data' and give the latest news. Also, what is AAPL's balance sheet information for 2023?"""
response = invoke_agent_helper(query, session_id, supervisor_agent_id, supervisor_agent_alias_id, enable_trace=True)
print(response)

[2024-12-17 19:52:26,962] p90570 {agents.py:760} INFO - invoke_agent response metadata:
[2024-12-17 19:52:26,962] p90570 {agents.py:761} INFO - {
  "RequestId": "ca406c51-a2e6-433f-9792-a3045551ef10",
  "HTTPStatusCode": 200,
  "HTTPHeaders": {
    "date": "Wed, 18 Dec 2024 00:52:26 GMT",
    "content-type": "application/vnd.amazon.eventstream",
    "transfer-encoding": "chunked",
    "connection": "keep-alive",
    "x-amzn-requestid": "ca406c51-a2e6-433f-9792-a3045551ef10",
    "x-amz-bedrock-agent-session-id": "59e00984-bcda-11ef-9404-168aeb12c117",
    "x-amzn-bedrock-agent-content-type": "application/json"
  },
  "RetryAttempts": 0
}
[2024-12-17 19:52:27,116] p90570 {agents.py:787} INFO - ==== Trace Event Received ====
[2024-12-17 19:52:27,116] p90570 {agents.py:790} INFO - callerChain: [
  {
    "agentAliasArn": "arn:aws:bedrock:us-east-1:218208277580:agent-alias/HM2GAT6PKT/K3WDMSOCAT"
  }
]
[2024-12-17 19:52:27,117] p90570 {agents.py:803} INFO - Orchestration Trace Detected
[2024

🍩 https://wandb.ai/madhur-prash-none/multi-agent-collaboration/r/call/0193d740-7867-7441-aa38-5479ea6eef94
Here is the latest news about Apple (AAPL):
- Apple reported better-than-anticipated Q4 revenue, but earnings were hurt by an EU tax payment.
- Apple CEO Tim Cook confirmed he has been personally holding cryptocurrency for the past 3 years.
- There are concerns that Apple's AI is causing the company big problems, according to data.
- Apple's stock price has turned positive for 2024 as the company addresses key bear concerns.
- Apple's stock hit a record high on optimism that AI will spur iPhone sales.
- Apple shares are back to winning ways as AI optimism builds around the company.

And here are the balance sheet highlights for Apple Inc. (AAPL) as of the fiscal year ending 2024-09-28:

Total Assets: $364,980,000,000
- Current Assets: $152,987,000,000
  - Cash and Equivalents: $29,943,000,000
  - Inventory: $7,286,000,000
  - Current Investments: $35,228,000,000
  - Receivables: $